# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [1]:
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from matplotlib.ticker import MaxNLocator

# read in the json files
portfolio_raw = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile_raw = pd.read_json('data/profile.json', orient='records', lines=True)
transcript_raw = pd.read_json('data/transcript.json', orient='records', lines=True)

## Data understanding



**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)


** -> Data is cleaned and ready to be used**

In [2]:
portfolio_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 612.0+ bytes


In [3]:
# portfolio_raw['channels'].explode().unique()

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**-> the NoneType gender will be removed from profile dataset and storage in a new dataset**

In [3]:
profile_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.2+ KB


**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

In [4]:
transcript_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   person  306534 non-null  object
 1   event   306534 non-null  object
 2   value   306534 non-null  object
 3   time    306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 9.4+ MB


In [5]:
transcript_raw['event'].value_counts()

event
transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: count, dtype: int64

In [6]:
{[*x][0] for x in transcript_raw['value']}

{'amount', 'offer id', 'offer_id'}

### Data Cleaning and Formatting Plan

#### **Objective**
- Clean and standardize the dataset while handling null values appropriately.

---

#### **Profile Dataset**
1. Split the `profile` dataset into two subsets:
   - One containing rows where the `gender` is `none`.
   - Another containing rows with all other `gender` values.

2. Convert the `became_member_on` column to a standardized **datetime** format for consistency and easier analysis.

---

#### **Portfolio Dataset**
- Update column names to make them more descriptive and easier to read.

---

#### **Transcript Dataset**
- Modify the keys within the `value` dictionaries by replacing spaces (`' '`) with underscores (`'_'`) to ensure uniformity and avoid potential issues in further processing.

---

In [7]:
profile = profile_raw.copy(deep=True)

# Convert the 'became_member_on' column to a datetime format
profile['became_member_on'] = pd.to_datetime(profile['became_member_on'], format='%Y%m%d')

# Create a new column with only the year and month of the membership
profile['bec_memb_year_month'] = pd.to_datetime(profile['became_member_on'], format='%Y%m%d').dt.strftime('%Y-%m')

In [ ]:
profile.info()

In [8]:
portfolio = portfolio_raw.copy()
portfolio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 612.0+ bytes


In [292]:
# Standardisation of column names
transcript = transcript_raw.copy(deep=True)
transcript['event'] = transcript['event'].str.replace(' ','_')

# Fixing the dicts key names
def fix_offer_id(value):
    if isinstance(value, dict) and 'offer id' in value:
        value['offer_id'] = value.pop('offer id')
    return value

transcript['value'] = transcript['value'].apply(fix_offer_id)

# Normalizing the value column and create a new dataframe
value_df = pd.json_normalize(transcript['value']) 
transcript_b = pd.concat([transcript, value_df], axis=1).drop('value', axis=1)

port_id = {
    'ae264e3637204a6fb9bb56bc8210ddfd': 'ofr_A',
    '4d5c57ea9a6940dd891ad53e9dbe8da0': 'ofr_B',
    '3f207df678b143eea3cee63160fa8bed': 'ofr_C',
    '9b98b8c7a33c4b65b9aebfe6a799e6d9': 'ofr_D',
    '0b1e1539f2cc45b7b9fa7c272da2e1d7': 'ofr_E',
    '2298d6c36e964ae4a3e7e9706d1fb8c2': 'ofr_F',
    'fafdcd668e3743c1bb461111dcafc2a4': 'ofr_G',
    '5a8bc65990b245e5a138643cd4eb9837': 'ofr_H',
    'f19421c1d4aa40978ebb69ca19b0e20d': 'ofr_I',
    '2906b810c7d4411798c6938adc9daaa5': 'ofr_J'
}

transcript_b['ofr_id_short'] = transcript_b['offer_id'].map(port_id)
transcript_b['ofr_id_short'] = transcript_b['ofr_id_short'].fillna('tran')


transactions = transcript_b[transcript_b['event'] == 'transaction'].copy()
events = transcript_b[transcript_b['event'] != 'transaction'].copy()

events['tag'] = (
    events.sort_values(by=['time'], ascending=True).groupby(['person', 'event', 'ofr_id_short'])
    .cumcount()
)

events.info()

<class 'pandas.core.frame.DataFrame'>
Index: 167581 entries, 0 to 306527
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   person        167581 non-null  object 
 1   event         167581 non-null  object 
 2   time          167581 non-null  int64  
 3   offer_id      167581 non-null  object 
 4   amount        0 non-null       float64
 5   reward        33579 non-null   float64
 6   ofr_id_short  167581 non-null  object 
 7   tag           167581 non-null  int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 11.5+ MB


In [254]:
events

,person,event,time,offer_id,amount,reward,ofr_id_short,tag
0,78afa995795e4d85b5d9ceeca43f5fef,offer_received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,ofr_D,0
1,a03223e636434f42ac4c3df47e8bac43,offer_received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,ofr_E,0
2,e2127556f4f64592b11af22de27a7932,offer_received,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,ofr_J,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer_received,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN,ofr_G,0
4,68617ca6246f4fbc85e91a2a49552598,offer_received,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN,ofr_B,0
...,...,...,...,...,...,...,...,...
306497,a6f84f4e976f44508c358cc9aba6d2b3,offer_completed,714,2298d6c36e964ae4a3e7e9706d1fb8c2,NaN,3.0,ofr_F,0
306506,b895c57e8cd047a8872ce02aa54759d6,offer_completed,714,fafdcd668e3743c1bb461111dcafc2a4,NaN,2.0,ofr_G,1
306507,8dda575c2a1d44b9ac8e8b07b93d1f8e,offer_viewed,714,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,ofr_E,0
306509,8431c16f8e1d440880db371a68f82dd0,offer_completed,714,fafdcd668e3743c1bb461111dcafc2a4,NaN,2.0,ofr_G,0


In [255]:
len(events['time'].value_counts())

120

In [256]:
events[events['person'] == '0009655768c64bdeb2e877511632db8f']

,person,event,time,offer_id,amount,reward,ofr_id_short,tag
55972,0009655768c64bdeb2e877511632db8f,offer_received,168,5a8bc65990b245e5a138643cd4eb9837,NaN,NaN,ofr_H,0
77705,0009655768c64bdeb2e877511632db8f,offer_viewed,192,5a8bc65990b245e5a138643cd4eb9837,NaN,NaN,ofr_H,0
113605,0009655768c64bdeb2e877511632db8f,offer_received,336,3f207df678b143eea3cee63160fa8bed,NaN,NaN,ofr_C,0
139992,0009655768c64bdeb2e877511632db8f,offer_viewed,372,3f207df678b143eea3cee63160fa8bed,NaN,NaN,ofr_C,0
153401,0009655768c64bdeb2e877511632db8f,offer_received,408,f19421c1d4aa40978ebb69ca19b0e20d,NaN,NaN,ofr_I,0
168413,0009655768c64bdeb2e877511632db8f,offer_completed,414,f19421c1d4aa40978ebb69ca19b0e20d,NaN,5.0,ofr_I,0
187554,0009655768c64bdeb2e877511632db8f,offer_viewed,456,f19421c1d4aa40978ebb69ca19b0e20d,NaN,NaN,ofr_I,0
204340,0009655768c64bdeb2e877511632db8f,offer_received,504,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN,ofr_G,0
228423,0009655768c64bdeb2e877511632db8f,offer_completed,528,fafdcd668e3743c1bb461111dcafc2a4,NaN,2.0,ofr_G,0
233413,0009655768c64bdeb2e877511632db8f,offer_viewed,540,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN,ofr_G,0


In [257]:
dfg = events.groupby(['person', 'event', 'ofr_id_short', 'tag']).agg({'time': 'max'}).unstack(2).reset_index()    #
dfg.columns = ['_'.join(col).strip('_') for col in dfg.columns.to_flat_index()]

dfg[dfg['person'] == '0009655768c64bdeb2e877511632db8f'].fillna('-').sort_values('event', ascending=True)

,person,event,tag,time_ofr_A,time_ofr_B,time_ofr_C,time_ofr_D,time_ofr_E,time_ofr_F,time_ofr_G,time_ofr_H,time_ofr_I,time_ofr_J
0,0009655768c64bdeb2e877511632db8f,offer_completed,0,-,-,-,-,-,-,528.0,-,414.0,576.0
1,0009655768c64bdeb2e877511632db8f,offer_received,0,-,-,336.0,-,-,-,504.0,168.0,408.0,576.0
2,0009655768c64bdeb2e877511632db8f,offer_viewed,0,-,-,372.0,-,-,-,540.0,192.0,456.0,-


In [ ]:
.groupby(['person', 'event', 'ofr_id_short', 'tag']).agg({'time': 'max'}).unstack(1).reset_index()dfg = events    #
dfg.columns = ['_'.join(col).strip('_') for col in dfg.columns.to_flat_index()]

dfg[dfg['person'] == '0009655768c64bdeb2e877511632db8f'].fillna('-') #.sort_values('tag', ascending=True)

,person,ofr_id_short,tag,time_offer_completed,time_offer_received,time_offer_viewed
0,0009655768c64bdeb2e877511632db8f,ofr_C,0,-,336.0,372.0
1,0009655768c64bdeb2e877511632db8f,ofr_G,0,528.0,504.0,540.0
2,0009655768c64bdeb2e877511632db8f,ofr_H,0,-,168.0,192.0
3,0009655768c64bdeb2e877511632db8f,ofr_I,0,414.0,408.0,456.0
4,0009655768c64bdeb2e877511632db8f,ofr_J,0,576.0,576.0,-


In [277]:
transactions[transactions['person'] == '0009655768c64bdeb2e877511632db8f'] #.sort_values('tag', ascending=True)

,person,event,time,offer_id,amount,reward,ofr_id_short
89291,0009655768c64bdeb2e877511632db8f,transaction,228,NaN,22.16,NaN,tran
168412,0009655768c64bdeb2e877511632db8f,transaction,414,NaN,8.57,NaN,tran
228422,0009655768c64bdeb2e877511632db8f,transaction,528,NaN,14.11,NaN,tran
237784,0009655768c64bdeb2e877511632db8f,transaction,552,NaN,13.56,NaN,tran
258883,0009655768c64bdeb2e877511632db8f,transaction,576,NaN,10.27,NaN,tran
293497,0009655768c64bdeb2e877511632db8f,transaction,660,NaN,12.36,NaN,tran
300930,0009655768c64bdeb2e877511632db8f,transaction,690,NaN,28.16,NaN,tran
302205,0009655768c64bdeb2e877511632db8f,transaction,696,NaN,18.41,NaN,tran


In [278]:
df_merged = events.merge(transactions, left_on=['person', 'time'], right_on=['person', 'time'], how='left').fillna('-')

In [279]:
transactions

,person,event,time,offer_id,amount,reward,ofr_id_short
12654,02c083884c7d45b39cc68e1314fec56c,transaction,0,NaN,0.83,NaN,tran
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,0,NaN,34.56,NaN,tran
12659,54890f68699049c2a04d415abc25e717,transaction,0,NaN,13.23,NaN,tran
12670,b2f1cd155b864803ad8334cdf13c4bd2,transaction,0,NaN,19.51,NaN,tran
12671,fe97aa22dd3e48c8b143116a8403dd52,transaction,0,NaN,18.97,NaN,tran
...,...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,714,NaN,1.59,NaN,tran
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,714,NaN,9.53,NaN,tran
306531,a00058cf10334a308c68e7631c529907,transaction,714,NaN,3.61,NaN,tran
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,714,NaN,3.53,NaN,tran


In [280]:
df_merged[df_merged['person'] == '0009655768c64bdeb2e877511632db8f'] #.drop(columns=['offer_id_x', 'amount_x', 'reward_y', 'ofr_id_short_y'])

,person,event_x,time,offer_id_x,amount_x,reward_x,ofr_id_short_x,tag,event_y,offer_id_y,amount_y,reward_y,ofr_id_short_y
30049,0009655768c64bdeb2e877511632db8f,offer_received,168,5a8bc65990b245e5a138643cd4eb9837,-,-,ofr_H,0,-,-,-,-,-
47320,0009655768c64bdeb2e877511632db8f,offer_viewed,192,5a8bc65990b245e5a138643cd4eb9837,-,-,ofr_H,0,-,-,-,-,-
58310,0009655768c64bdeb2e877511632db8f,offer_received,336,3f207df678b143eea3cee63160fa8bed,-,-,ofr_C,0,-,-,-,-,-
77536,0009655768c64bdeb2e877511632db8f,offer_viewed,372,3f207df678b143eea3cee63160fa8bed,-,-,ofr_C,0,-,-,-,-,-
83625,0009655768c64bdeb2e877511632db8f,offer_received,408,f19421c1d4aa40978ebb69ca19b0e20d,-,-,ofr_I,0,-,-,-,-,-
96948,0009655768c64bdeb2e877511632db8f,offer_completed,414,f19421c1d4aa40978ebb69ca19b0e20d,-,5.0,ofr_I,0,transaction,-,8.57,-,tran
105987,0009655768c64bdeb2e877511632db8f,offer_viewed,456,f19421c1d4aa40978ebb69ca19b0e20d,-,-,ofr_I,0,-,-,-,-,-
112507,0009655768c64bdeb2e877511632db8f,offer_received,504,fafdcd668e3743c1bb461111dcafc2a4,-,-,ofr_G,0,-,-,-,-,-
130750,0009655768c64bdeb2e877511632db8f,offer_completed,528,fafdcd668e3743c1bb461111dcafc2a4,-,2.0,ofr_G,0,transaction,-,14.11,-,tran
132869,0009655768c64bdeb2e877511632db8f,offer_viewed,540,fafdcd668e3743c1bb461111dcafc2a4,-,-,ofr_G,0,-,-,-,-,-


In [282]:
transactions[transactions['person'] == 'b895c57e8cd047a8872ce02aa54759d6']

,person,event,time,offer_id,amount,reward,ofr_id_short
18031,b895c57e8cd047a8872ce02aa54759d6,transaction,6,NaN,0.97,NaN,tran
22271,b895c57e8cd047a8872ce02aa54759d6,transaction,18,NaN,2.64,NaN,tran
26099,b895c57e8cd047a8872ce02aa54759d6,transaction,30,NaN,1.95,NaN,tran
35521,b895c57e8cd047a8872ce02aa54759d6,transaction,66,NaN,0.72,NaN,tran
45410,b895c57e8cd047a8872ce02aa54759d6,transaction,114,NaN,4.70,NaN,tran
46584,b895c57e8cd047a8872ce02aa54759d6,transaction,120,NaN,0.97,NaN,tran
53152,b895c57e8cd047a8872ce02aa54759d6,transaction,162,NaN,1.08,NaN,tran
95079,b895c57e8cd047a8872ce02aa54759d6,transaction,246,NaN,0.19,NaN,tran
107970,b895c57e8cd047a8872ce02aa54759d6,transaction,312,NaN,0.48,NaN,tran
201546,b895c57e8cd047a8872ce02aa54759d6,transaction,498,NaN,0.44,NaN,tran


In [281]:

df_merged[df_merged['person'] == 'b895c57e8cd047a8872ce02aa54759d6']

,person,event_x,time,offer_id_x,amount_x,reward_x,ofr_id_short_x,tag,event_y,offer_id_y,amount_y,reward_y,ofr_id_short_y
12466,b895c57e8cd047a8872ce02aa54759d6,offer_received,0,fafdcd668e3743c1bb461111dcafc2a4,-,-,ofr_G,0,-,-,-,-,-
16610,b895c57e8cd047a8872ce02aa54759d6,offer_viewed,6,fafdcd668e3743c1bb461111dcafc2a4,-,-,ofr_G,0,transaction,-,0.97,-,tran
26220,b895c57e8cd047a8872ce02aa54759d6,offer_completed,114,fafdcd668e3743c1bb461111dcafc2a4,-,2.0,ofr_G,0,transaction,-,4.7,-,tran
68056,b895c57e8cd047a8872ce02aa54759d6,offer_received,336,5a8bc65990b245e5a138643cd4eb9837,-,-,ofr_H,0,-,-,-,-,-
78138,b895c57e8cd047a8872ce02aa54759d6,offer_viewed,372,5a8bc65990b245e5a138643cd4eb9837,-,-,ofr_H,0,-,-,-,-,-
93417,b895c57e8cd047a8872ce02aa54759d6,offer_received,408,4d5c57ea9a6940dd891ad53e9dbe8da0,-,-,ofr_B,0,-,-,-,-,-
108238,b895c57e8cd047a8872ce02aa54759d6,offer_viewed,474,4d5c57ea9a6940dd891ad53e9dbe8da0,-,-,ofr_B,0,-,-,-,-,-
122271,b895c57e8cd047a8872ce02aa54759d6,offer_received,504,fafdcd668e3743c1bb461111dcafc2a4,-,-,ofr_G,1,transaction,-,0.68,-,tran
129089,b895c57e8cd047a8872ce02aa54759d6,offer_viewed,516,fafdcd668e3743c1bb461111dcafc2a4,-,-,ofr_G,1,transaction,-,1.44,-,tran
167577,b895c57e8cd047a8872ce02aa54759d6,offer_completed,714,fafdcd668e3743c1bb461111dcafc2a4,-,2.0,ofr_G,1,transaction,-,4.48,-,tran


In [284]:
dfg[dfg['person'] == 'b895c57e8cd047a8872ce02aa54759d6'].fillna('-') #.sort_values('tag', ascending=True)

,person,ofr_id_short,tag,time_offer_completed,time_offer_received,time_offer_viewed
55449,b895c57e8cd047a8872ce02aa54759d6,ofr_B,0,-,408.0,474.0
55450,b895c57e8cd047a8872ce02aa54759d6,ofr_G,0,114.0,0.0,6.0
55451,b895c57e8cd047a8872ce02aa54759d6,ofr_G,1,714.0,504.0,516.0
55452,b895c57e8cd047a8872ce02aa54759d6,ofr_H,0,-,336.0,372.0


In [293]:
transcript_b

,person,event,time,offer_id,amount,reward,ofr_id_short
0,78afa995795e4d85b5d9ceeca43f5fef,offer_received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,ofr_D
1,a03223e636434f42ac4c3df47e8bac43,offer_received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,ofr_E
2,e2127556f4f64592b11af22de27a7932,offer_received,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,ofr_J
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer_received,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN,ofr_G
4,68617ca6246f4fbc85e91a2a49552598,offer_received,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN,ofr_B
...,...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,714,NaN,1.59,NaN,tran
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,714,NaN,9.53,NaN,tran
306531,a00058cf10334a308c68e7631c529907,transaction,714,NaN,3.61,NaN,tran
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,714,NaN,3.53,NaN,tran


In [313]:
transcript_b['tag'] = (
    transcript_b.sort_values(by=['time'], ascending=True).groupby(['person', 'event', 'ofr_id_short'])
    .cumcount()
)

transcript_b[transcript_b['person'] =='b895c57e8cd047a8872ce02aa54759d6'].groupby(['person', 'event', 'ofr_id_short', 'tag', 'time']).agg({'time': 'max'}).unstack(1).fillna('-').reset_index()#.sort_values(by='time', ascending=True)

person ofr_id_short tag time                  \
event                                                         offer_completed   
0      b895c57e8cd047a8872ce02aa54759d6        ofr_B   0  408               -   
1      b895c57e8cd047a8872ce02aa54759d6        ofr_B   0  474               -   
2      b895c57e8cd047a8872ce02aa54759d6        ofr_G   0    0               -   
3      b895c57e8cd047a8872ce02aa54759d6        ofr_G   0    6               -   
4      b895c57e8cd047a8872ce02aa54759d6        ofr_G   0  114           114.0   
5      b895c57e8cd047a8872ce02aa54759d6        ofr_G   1  504               -   
6      b895c57e8cd047a8872ce02aa54759d6        ofr_G   1  516               -   
7      b895c57e8cd047a8872ce02aa54759d6        ofr_G   1  714           714.0   
8      b895c57e8cd047a8872ce02aa54759d6        ofr_H   0  336               -   
9      b895c57e8cd047a8872ce02aa54759d6        ofr_H   0  372               -   
10     b895c57e8cd047a8872ce02aa54759d6         tran   0    6               -   
11     b895c57e8cd047a8872ce02aa54759d6         tran   1   18               -   
12     b895c57e8cd047a8872ce02aa54759d6         tran   2   30               -   
13     b895c57e8cd047a8872ce02aa54759d6         tran   3   66               -   
14     b895c57e8cd047a8872ce02aa54759d6         tran   4  114               -   
15     b895c57e8cd047a8872ce02aa54759d6         tran   5  120               -   
16     b895c57e8cd047a8872ce02aa54759d6         tran   6  162               -   
17     b895c57e8cd047a8872ce02aa54759d6         tran   7  246               -   
18     b895c57e8cd047a8872ce02aa54759d6         tran   8  312               -   
19     b895c57e8cd047a8872ce02aa54759d6         tran   9  498               -   
20     b895c57e8cd047a8872ce02aa54759d6         tran  10  504               -   
21     b895c57e8cd047a8872ce02aa54759d6         tran  11  516               -   
22     b895c57e8cd047a8872ce02aa54759d6         tran  12  618               -   
23     b895c57e8cd047a8872ce02aa54759d6         tran  13  660               -   
24     b895c57e8cd047a8872ce02aa54759d6         tran  14  690               -   
25     b895c57e8cd047a8872ce02aa54759d6         tran  15  714               -   

                                               
event offer_received offer_viewed transaction  
0              408.0            -           -  
1                  -        474.0           -  
2                0.0            -           -  
3                  -          6.0           -  
4                  -            -           -  
5              504.0            -           -  
6                  -        516.0           -  
7                  -            -           -  
8              336.0            -           -  
9                  -        372.0           -  
10                 -            -         6.0  
11                 -            -        18.0  
12                 -            -        30.0  
13                 -            -        66.0  
14                 -            -       114.0  
15                 -            -       120.0  
16                 -            -       162.0  
17                 -            -       246.0  
18                 -            -       312.0  
19                 -            -       498.0  
20                 -            -       504.0  
21                 -            -       516.0  
22                 -            -       618.0  
23                 -            -       660.0  
24                 -            -       690.0  
25                 -            -       714.0

In [289]:
transcript_b['ofr_id_short'].value_counts()

ofr_id_short
tran     138953
ofr_G     20241
ofr_F     20139
ofr_I     19131
ofr_B     18222
ofr_A     18062
ofr_D     16202
ofr_J     15767
ofr_H     14305
ofr_E     13751
ofr_C     11761
Name: count, dtype: int64